In [2]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable


In [10]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--

In [39]:
# basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing & ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
#xgboost
import xgboost as xgb
from xgboost import XGBClassifier  # or XGBRegressor for regression tasks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# streamlit
import streamlit as st

# Classification: Conversion Prediction
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter

In [48]:
df1 = pd.read_csv('C:/Users/bhuva/Downloads/train_data (1).csv')
df2 = pd.read_csv('C:/Users/bhuva/Downloads/test_data (1).csv')

In [49]:
#Preprocessing 
def preprocess_data(df1):
    # Fill missing numerical values
    num_cols = df1.select_dtypes(include=['int64', 'float64']).columns
    for col in num_cols:
        df1[col].fillna(df1[col].median())

    # Fill missing categorical values
    cat_cols = df1.select_dtypes(include='object').columns
    for col in cat_cols:
        df1[col].fillna(df1[col].mode()[0])

    # Label Encoding for categorical columns
    le = LabelEncoder()
    for col in cat_cols:
        df1[col] = le.fit_transform(df1[col])

    # Feature Engineering
    df1['session_date'] = pd.to_datetime(df1[['year', 'month', 'day']])
    df1['session_duration'] = df1.groupby('session_id')['page'].transform('count')
    df1['avg_price'] = (df1['price'] + df1['price_2']) / 2

    # Drop unnecessary columns
    df1.drop(columns=['year', 'month', 'day', 'session_date', 'session_id'], inplace=True)

    return df1

In [50]:
#Classification: Conversion Prediction
def train_classification_model(df1):
    df1 = preprocess_data(df1)
    
    X = df1.drop(columns=['order'])  # target is 'order' - converted or not
    y = df1['order']

    # Encode labels if not numeric
    if y.dtype == 'object':
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)
    else:
        label_encoder = None

     # Handle class imbalance
    class_counts = Counter(y)
    min_class_size = min(class_counts.values())

    if min_class_size < 2:
        st.warning("SMOTE skipped: One of the classes has fewer than 2 samples. Consider collecting more data.")
        X_resampled, y_resampled = X, y
    else:
        k_neighbors = min(5, min_class_size - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_resampled, y_resampled = smote.fit_resample(X, y)

    # Balance dataset
   # smote = SMOTE(k_neighbors=k_neighbors)
    #X_resampled, y_resampled = smote.fit_resample(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted',zero_division=0),
        'Recall': recall_score(y_test, y_pred,average='weighted',zero_division=0),
        'F1 Score': f1_score(y_test, y_pred,average='weighted',zero_division=0),
    }

    return model, metrics, label_encoder

In [58]:
#Regression: Revenue Prediction
def train_regression_model(df1):
    df1 = preprocess_data(df1)
    
    df1['revenue'] = df1['price'] * df1['order']  # create a synthetic revenue feature
    
    X = df1.drop(columns=['revenue'])
    y = df1['revenue']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = {
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R-squared': r2_score(y_test, y_pred)
    }

    return model, metrics

In [59]:
#Clustering: Customer Segmentation
def train_clustering_model(df1):
    df1 = preprocess_data(df1)
    
    X = df1.drop(columns=['order'])  # Remove label
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    kmeans = KMeans(n_clusters=3, random_state=42)
    labels = kmeans.fit_predict(X_scaled)

    metrics = {
        'Silhouette Score': silhouette_score(X_scaled, labels),
        'Davies-Bouldin Index': davies_bouldin_score(X_scaled, labels)
    }

    df1['Cluster'] = labels
    return df1[['Cluster']], metrics

In [60]:
# Streamlit Interface
st.title("🛒 Customer Conversion Analysis using Clickstream Data")

uploaded_file = st.file_uploader("Upload your CSV file", type=["csv"])

if uploaded_file:
    df1 = pd.read_csv(uploaded_file)
    st.success("File Uploaded Successfully!")

    option = st.selectbox("Choose a Task", ["Classification", "Regression", "Clustering"])

    if option == "Classification":
        model, metrics = train_classification_model(df1)
        st.subheader("📈 Classification Results")
        for k, v in metrics.items():
            st.write(f"{k}: {v:.2f}")

    elif option == "Regression":
        model, metrics = train_regression_model(df1)
        st.subheader("📉 Regression Results")
        for k, v in metrics.items():
            st.write(f"{k}: {v:.2f}")

    elif option == "Clustering":
        clustered_df, metrics = train_clustering_model(df1)
        st.subheader("📊 Clustering Results")
        st.write(clustered_df.head())
        for k, v in metrics.items():
            st.write(f"{k}: {v:.2f}")

2025-04-15 22:18:44.002 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 22:18:44.619 
  command:

    streamlit run C:\Users\bhuva\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-15 22:18:44.620 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 22:18:44.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 22:18:44.624 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 22:18:44.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 22:18:44.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 22:18:44.636 Thre